In [ ]:
# Cell 1: 載入硬體
from pynq import Overlay
from pynq import allocate
import numpy as np
import time

# 載入 Bitstream (改成自己的檔名)
overlay = Overlay("example.bit")


cdma = overlay.axi_cdma_0
bram_ctrl = overlay.axi_bram_ctrl_0 

print("Bitstream loaded and IP objects created.")

In [ ]:
# Cell 2: 定義地址與暫存器偏移量

CDMA_BASE_ADDR = 0x7E200000  # CDMA 的 Control 介面地址
BRAM_BASE_ADDR = 0x40000000  # BRAM 的物理地址 (目的地)

# CDMA 暫存器偏移量 (Register Map)
CDMACR  = 0x00  # Control Register
CDMASR  = 0x04  # Status Register
SA_REG  = 0x18  # Source Address
DA_REG  = 0x20  # Destination Address
BTT_REG = 0x28  # Bytes to Transfer

print(f"Configuration Set:\n - CDMA Reg Base: {hex(CDMA_BASE_ADDR)}\n - BRAM Phys Addr: {hex(BRAM_BASE_ADDR)}")

In [ ]:
# Cell 3: 定義輔助函式

def get_cdma_status():
    """讀取並解析 CDMA 狀態暫存器"""
    status = cdma.read(CDMASR)
    print(f"Current Status Register (SR): {hex(status)}")
    if status & 0x02: print(" - Idle (Ready)")
    else:             print(" - Busy (Working)")
    if status & 0x10: print(" - IRQ: Internal Error")
    if status & 0x20: print(" - IRQ: Slave Error (Target refused access)")
    if status & 0x40: print(" - IRQ: Decode Error (Invalid Address)")
    if status & 0x1000: print(" - IRQ: IOC (Transfer Complete)")
    return status

def reset_cdma():
    """軟體重置 CDMA"""
    print("Resetting CDMA...")
    cdma.write(CDMACR, 0x04) # 寫入 Reset bit
    # 等待 Reset 完成
    timeout = 5
    while (cdma.read(CDMACR) & 0x04) and timeout > 0:
        time.sleep(0.1)
        timeout -= 1
    
    if timeout == 0:
        print("Warning: Reset timeout!")
    else:
        print("CDMA Reset Done.")

get_cdma_status()

In [ ]:
# Cell 4: 準備測試數據

DATA_LEN = 10  # 測試 10 個整數
BUFFER_SIZE = DATA_LEN * 4 # 每個 int 4 bytes

# 使用 PYNQ allocate 分配連續記憶體 (CMA)
input_buffer = allocate(shape=(DATA_LEN,), dtype=np.uint32)

# 初始化數據 (填入 10, 20, 30...)
for i in range(DATA_LEN):
    input_buffer[i] = (i + 1) * 10

# 將 Cache 數據寫入物理記憶體 (DDR)
input_buffer.flush()

print(f"Input Buffer Allocated.")
print(f" - Physical Address: {hex(input_buffer.device_address)} (給 CDMA 用的)")
print(f" - Data: {input_buffer}")

In [ ]:
# Cell 5: 執行搬運 (DDR -> BRAM)
print("=== Starting CDMA Transfer ===")

reset_cdma()

# 設定來源地址 (使用 input_buffer 的物理地址)
src_phys = input_buffer.device_address
cdma.write(SA_REG, src_phys)
print(f"Written SA: {hex(src_phys)}")

# 設定目的地址 (BRAM 的物理地址)
dst_phys = BRAM_BASE_ADDR
cdma.write(DA_REG, dst_phys)
print(f"Written DA: {hex(dst_phys)}")

# 設定長度 觸發傳輸
print(f"Triggering transfer of {BUFFER_SIZE} bytes...")
cdma.write(BTT_REG, BUFFER_SIZE)

# Polling
wait_cycles = 1000
while True:
    status = cdma.read(CDMASR)
    if status & 0x02: # 檢查 Bit 1 (Idle)
        break
    wait_cycles -= 1
    if wait_cycles == 0:
        print("Error: Transfer Timed Out!")
        break

final_status = get_cdma_status()
if final_status & 0x02 and not (final_status & 0x70):
    print("\n>>> Transfer Successful (Hardware View) <<<")
else:
    print("\n>>> Transfer Failed or Error Detected <<<")

In [ ]:
# Cell 6: 軟體驗證 (CPU Read Check)

print("=== Verifying Data in BRAM ===")
error_count = 0

for i in range(DATA_LEN):
    offset = i * 4

    data_read = bram_ctrl.read(offset)
    
    expected = input_buffer[i]
    
    if data_read != expected:
        print(f"Error at index {i}: Expected {expected}, Got {data_read}")
        error_count += 1
    else:
        if i < 5:
            print(f"Addr offset {hex(offset)}: {data_read} (Match)")

if error_count == 0:
    print("\nSUCCESS! All data matches.")
else:
    print(f"\nFAILED! Found {error_count} errors.")